In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer,LabelEncoder,StandardScaler,MinMaxScaler, OrdinalEncoder
from sklearn.linear_model import LogisticRegression,SGDClassifier,LinearRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBRegressor
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
import keras
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
import datetime
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split
from time import time

from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline


In [ ]:
train_df = pd.read_csv("../input/covid19-global-forecasting-week-4/train.csv")
test_df = pd.read_csv("../input/covid19-global-forecasting-week-4/test.csv")
submission = pd.read_csv("../input/covid19-global-forecasting-week-4/submission.csv")


In [ ]:
#Get the top 10 countries
last_date = train_df.Date.max()
df_countries = train_df[train_df['Date']==last_date]
df_countries = df_countries.groupby('Country_Region', as_index=False)['ConfirmedCases','Fatalities'].sum()
df_countries = df_countries.nlargest(10,'ConfirmedCases')
#Get the trend for top 10 countries
df_trend = train_df.groupby(['Date','Country_Region'], as_index=False)['ConfirmedCases','Fatalities'].sum()
df_trend = df_trend.merge(df_countries, on='Country_Region')
df_trend.drop(['ConfirmedCases_y','Fatalities_y'],axis=1, inplace=True)
df_trend.rename(columns={'Country_Region':'Country', 'ConfirmedCases_x':'Cases', 'Fatalities_x':'Deaths'}, inplace=True)
#Add columns for studying logarithmic trends
df_trend['ln(Cases)'] = np.log(df_trend['Cases']+1)# Added 1 to remove error due to log(0).
df_trend['ln(Deaths)'] = np.log(df_trend['Deaths']+1)

In [ ]:
px.line(df_trend, x='Date', y='Cases', color='Country', title='COVID19 Total Cases growth for top 10 worst affected countries')

In [ ]:
#Get the top 10 countries
last_date = train_df.Date.max()
df_countries = train_df[train_df['Date']==last_date]
df_countries = df_countries.groupby('Country_Region', as_index=False)['ConfirmedCases','Fatalities'].sum()
df_countries = df_countries.nsmallest(10,'ConfirmedCases')
#Get the trend for top 10 countries
df_trend = train_df.groupby(['Date','Country_Region'], as_index=False)['ConfirmedCases','Fatalities'].sum()
df_trend = df_trend.merge(df_countries, on='Country_Region')
df_trend.drop(['ConfirmedCases_y','Fatalities_y'],axis=1, inplace=True)
df_trend.rename(columns={'Country_Region':'Country', 'ConfirmedCases_x':'Cases', 'Fatalities_x':'Deaths'}, inplace=True)
#Add columns for studying logarithmic trends
df_trend['ln(Cases)'] = np.log(df_trend['Cases']+1)# Added 1 to remove error due to log(0).
df_trend['ln(Deaths)'] = np.log(df_trend['Deaths']+1)

In [ ]:
px.line(df_trend, x='Date', y='Cases', color='Country', title='COVID19 Total Cases growth for least 10 worst affected countries')

In [ ]:
columns = ['ln(No_of_Days)', 'Country_Region','ConfirmedCases','Fatalities']
test_set_columns = ['ln(No_of_Days)', 'Country_Region']

In [ ]:
def extract_features(df):
    
    #df['No_of_Days'] = df['Date'].apply(lambda x: relativedelta(datetime.datetime.strptime(x, '%Y-%m-%d'), datetime.datetime.strptime(min(df['Date']), '%Y-%m-%d')).days)
    df['month'] = df['Date'].apply(lambda x: int(x.split(' ')[0].split('-')[1]))
   
    df['day'] = df['Date'].apply(lambda x: int(x.split(' ')[0].split('-')[2]))
    df['is_weekend'] = ((df.Date.astype('datetime64[ns]').dt.dayofweek) // 4 == 1).astype(float)
    df['weekday'] = df.Date.astype('datetime64[ns]').dt.dayofweek
    df['is_holyday'] = df.apply(lambda row: 1 if (row['month']==1 and row['day']==1) or (row['month']==7 and row['day']==4) or (row['month']==11 and row['day']==11) or (row['month']==12 and row['day']==25) or (row['month']==1 and row['day'] >= 15 and row['day'] <= 21 and row['weekday'] == 0) or (row['month']==2 and row['day'] >= 15 and row['day'] <= 21 and row['weekday'] == 0) or (row['month']==5 and row['day'] >= 25 and row['day'] <= 31 and row['weekday'] == 0) or (row['month']==9 and row['day'] >= 1 and row['day'] <= 7 and row['weekday'] == 0) or (row['month']==10 and row['day'] >= 8 and row['day'] <= 14 and row['weekday'] == 0) or (row['month']==11 and row['day'] >= 22 and row['day'] <= 28 and row['weekday'] == 3) else 0, axis=1)
    df['is_day_before_holyday'] = df.apply(lambda row: 1 if (row['month']==12 and row['day']==31) or (row['month']==7 and row['day']==3) or (row['month']==11 and row['day']==10) or (row['month']==12 and row['day']==24) or (row['month']==1 and row['day'] >= 14 and row['day'] <= 20 and row['weekday'] == 6) or (row['month']==2 and row['day'] >= 14 and row['day'] <= 20 and row['weekday'] == 6) or (row['month']==5 and row['day'] >= 24 and row['day'] <= 30 and row['weekday'] == 6) or ((row['month']==9 and row['day'] >= 1 and row['day'] <= 6) or (row['month']==8 and row['day'] == 31) and row['weekday'] == 6) or (row['month']==10 and row['day'] >= 7 and row['day'] <= 13 and row['weekday'] == 6) or (row['month']==11 and row['day'] >= 21 and row['day'] <= 27 and row['weekday'] == 2) else 0, axis=1)
    #df.drop('Date', axis=1, inplace=True)
    #df.drop('month', axis=1, inplace=True)
    #df.drop('day', axis=1, inplace=True)

In [ ]:
train_df['Province_State'].fillna("",inplace = True)
test_df['Province_State'].fillna("",inplace = True)

In [ ]:
extract_features(train_df)
extract_features(test_df)

In [ ]:
for i in range(len(train_df)):
    if train_df["Province_State"][i] != '':
        train_df["Country_Region"][i] = train_df["Province_State"][i] + " (" + str(train_df["Country_Region"][i]) + ")"
    
for i in range(len(test_df)):
    if test_df["Province_State"][i] != '':
        test_df["Country_Region"][i] = test_df["Province_State"][i] + " (" + str(test_df["Country_Region"][i]) + ")"
        
train_df.drop(columns = "Province_State", inplace=True)
test_df.drop(columns = "Province_State", inplace=True)


In [ ]:
i = 0
for value in train_df["Country_Region"].unique():
    if i < len(train_df):
        j = 1
        while(train_df["Country_Region"][i] == value):
            train_df["day"][i] = j
            j += 1; i += 1
            if i == len(train_df):
                break

i = 0
for value in test_df["Country_Region"].unique():
    if i < len(test_df):
        j = 72
        while(test_df["Country_Region"][i] == value):
            test_df["day"][i] = j
            j += 1; i += 1
            if i == len(test_df):
                break
                
train_df.rename(columns = {"day" : "No_of_Days"}, inplace = True)
test_df.rename(columns = {"day" : "No_of_Days"}, inplace = True)

train_df['ln(No_of_Days)'] = np.log1p(train_df['No_of_Days'])
test_df['ln(No_of_Days)'] = np.log1p(test_df['No_of_Days'])

In [ ]:
#Get the top 10 countries
last_date = train_df.No_of_Days.max()
df_countries = train_df[train_df['No_of_Days']==last_date]
df_countries = df_countries.groupby('Country_Region', as_index=False)['ConfirmedCases','Fatalities'].sum()
df_countries = df_countries.nlargest(10,'ConfirmedCases')
#Get the trend for top 10 countries
df_trend = train_df.groupby(['No_of_Days','Country_Region'], as_index=False)['ConfirmedCases','Fatalities'].sum()
df_trend = df_trend.merge(df_countries, on='Country_Region')
df_trend.drop(['ConfirmedCases_y','Fatalities_y'],axis=1, inplace=True)
df_trend.rename(columns={'Country_Region':'Country', 'ConfirmedCases_x':'Cases', 'Fatalities_x':'Deaths'}, inplace=True)
#Add columns for studying logarithmic trends
df_trend['ln(Cases)'] = np.log1p(df_trend['Cases'])
df_trend['ln(Deaths)'] = np.log1p(df_trend['Deaths'])

In [ ]:
px.line(df_trend, x='No_of_Days', y='Cases', color='Country', title='COVID19 Total Cases growth for top 10 worst affected countries')

In [ ]:
train_df['ConfirmedCases'] = np.log1p(train_df['ConfirmedCases'])
train_df['Fatalities'] = np.log1p(train_df['Fatalities'])

train_df['ConfirmedCases'] = train_df['ConfirmedCases'].apply(int)
train_df['Fatalities'] = train_df['Fatalities'].apply(int)

In [ ]:
train_df

grid = {
    'C': np.linspace(0.01, 10),
    'epsilon': np.linspace(0.01, 10)
}
    
svr_gridsearch =  LinearSVR(fit_intercept=True, max_iter=10000)
grid_svr = GridSearchCV(svr_gridsearch, grid, scoring='neg_mean_absolute_error', cv=5)
grid_svr.fit(x_train, cases)

best_grid_svr_mae = grid_svr.best_estimator_
best_grid_svr_mae.fit(x_train, cases)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

best_grid_svr_mae.predict(x_test)

In [ ]:

df_train = train_df[columns]
df_test = test_df[test_set_columns]
#df_dev = df_dev[columns]




In [ ]:
submission = []
#Loop through all the unique countries
for country in df_train.Country_Region.unique():
    #Filter on the basis of country
    df_train1 = df_train[df_train["Country_Region"]==country]
    cases = np.array(df_train1.ConfirmedCases)
    fatalities = np.array(df_train1.Fatalities)
    del df_train1['ConfirmedCases']
    del df_train1['Fatalities']  
    
    lb = LabelEncoder()
    df_train1['Country_Region'] = lb.fit_transform(df_train1['Country_Region'])
  
    #Convert to numpy array for training
    #train = df_train1.values
    #sc = StandardScaler()
    scaler = MinMaxScaler()
    train = scaler.fit_transform(df_train1.values)
    #train = sc.fit_transform(train)
    #Separate the features and labels
    X_train, y_train = train, cases
    #model1 for predicting Confirmed Cases
    param_dict = {
              'n_estimators': [1500, 2000, 2200, 2500, 3000],
              'max_depth': [10,15,20, 25, 30],
              'min_child_weight': [1,2,3,4, 5, 6, 7, 8, 9, 10],
              'learning_rate': [0.05, 0.1,0.15, 0.2, 0.3, 0.5],
              'gamma': [0.0, 0.1, 0.2, 0.3, 0.4]
    }
    
    model_xgb_c = XGBRegressor(gamma = 0.4, learning_rate = 0.05, max_depth = 10, min_child_weight= 7, n_estimators = 1500, random_state = 37 ) 
    polynomial_features_c= PolynomialFeatures(degree=2)
    X_train_poly = polynomial_features_c.fit_transform(X_train)
    
    #steps1 = [('poly_features', PolynomialFeatures(3, interaction_only=True)),
    #     ('XGBoost', XGBRegressor(gamma = 0.4, learning_rate = 0.05, max_depth = 10, min_child_weight= 7, n_estimators = 1500, random_state = 37 ))]
    steps1 = [('poly_features', PolynomialFeatures(4, interaction_only=True)),
         ('lg', LinearRegression())]

    pipeline1 = Pipeline(steps=steps1)

    #start = time()
    #grid_search1 = GridSearchCV(model1, param_dict)
    #grid_search1.fit(X_train, y_train)
    #print("GridSearch took %.2f seconds to complete." % (time()-start))
    #display(grid_search1.best_params_)
    
    #l1 = LinearRegression()
    #r2 = RandomForestRegressor(n_estimators=150, random_state=37)
    #model1 = BaggingRegressor(base_estimator=x1, n_estimators=500, random_state=37, verbose=1, n_jobs= -1)
    #model1 = VotingRegressor([('xgb', x1), ('lr', l1), ('rf', r2)], n_jobs = -1)
    pipeline1.fit(X_train, y_train)

    #model2 for predicting Fatalities

    x_train_cas = []
    for i in range(len(X_train)):
        x = list(X_train[i])
        x.append(y_train[i])
        x_train_cas.append(x)
    x_train_cas[0]

    model2 = XGBRegressor(gamma = 0.4,
                             learning_rate = 0.05,
                             max_depth = 10,
                             min_child_weight= 7,
                             n_estimators = 1500, random_state = 37 )
    
    steps2 = [('poly_features', PolynomialFeatures(4, interaction_only=True)),
         ('XGBoost', LinearRegression())]

    pipeline2 = Pipeline(steps=steps2)

    #start = time()
    #grid_search2 = GridSearchCV(model2, param_dict)
    #grid_search2.fit(np.array(x_train_cas), fatalities)
    #print("GridSearch took %.2f seconds to complete." % (time()-start))
    #display(grid_search2.best_params_)
    
    
    
    
    #l21 = LinearRegression()
    #r22 = RandomForestRegressor(n_estimators=150, random_state=37)
    #model2 = BaggingRegressor(base_estimator=x21, n_estimators=500, random_state=37, verbose=1, n_jobs= -1)
    #model2 = VotingRegressor([('xgb', x21), ('lr', l21), ('rf', r22)], n_jobs = -1)
    
    #polynomial_features_f= PolynomialFeatures(degree=2)
    #x_train_cas_poly = polynomial_features_f.fit_transform(np.array(x_train_cas))
    pipeline2.fit(np.array(x_train_cas), fatalities)

    #Get the test data for that particular country and state
    df_test1 = test_df[(test_df["Country_Region"]==country)]
    df_test1['Country_Region'] = lb.transform(df_test1['Country_Region'])
    #Store the ForecastId separately
    ForecastId = df_test1.ForecastId.values
    #Remove the unwanted columns
    df_test1 = df_test1[test_set_columns]
    #df_test2 = sc.transform(df_test2)
    #Get the predictions
    
    df_test1_scaled = scaler.transform(df_test1.values)
    cases_pred = pipeline1.predict(df_test1_scaled)
    
    cases_pred[cases_pred < 0] = 0
    
    x_test_cas = []
    for i in range(len(df_test1_scaled)):
        x = list(df_test1_scaled[i])
        x.append(cases_pred[i])
        x_test_cas.append(x)
    x_test_cas[0]
    
    fatalities_pred = pipeline2.predict(np.array(x_test_cas))
    
    fatalities_pred[fatalities_pred < 0] = 0
    #Append the predicted values to submission list
    for i in range(len(cases_pred)):
        d = {'ForecastId':ForecastId[i], 'ConfirmedCases':np.around(np.expm1(cases_pred[i]),decimals = 0), 'Fatalities':np.around(np.expm1(fatalities_pred[i]),decimals = 0)}
        submission.append(d)


In [ ]:
df_submit = pd.DataFrame(submission)
df_submit.to_csv(r'submission.csv', index=False)

In [ ]:
output_df = pd.merge(test_df, df_submit, on='ForecastId')


In [ ]:
output_df['Country_Region'].unique()

In [ ]:
output_df[output_df['Country_Region'] == 'India']

In [ ]:
#Get the top 10 countries
last_date = output_df.Date.max()
df_countries = output_df[output_df['Date']==last_date]
df_countries = df_countries.groupby('Country_Region', as_index=False)['ConfirmedCases','Fatalities'].sum()
df_countries = df_countries.nlargest(20,'ConfirmedCases')
#Get the trend for top 10 countries
df_trend = output_df.groupby(['Date','Country_Region'], as_index=False)['ConfirmedCases','Fatalities'].sum()
df_trend = df_trend.merge(df_countries, on='Country_Region')
df_trend.drop(['ConfirmedCases_y','Fatalities_y'],axis=1, inplace=True)
df_trend.rename(columns={'Country_Region':'Country', 'ConfirmedCases_x':'Cases', 'Fatalities_x':'Deaths'}, inplace=True)
#Add columns for studying logarithmic trends
df_trend['ln(Cases)'] = np.log(df_trend['Cases']+1)# Added 1 to remove error due to log(0).
df_trend['ln(Deaths)'] = np.log(df_trend['Deaths']+1)

In [ ]:
px.line(df_trend, x='Date', y='Cases', color='Country', title='COVID19 Total Cases growth for top 10 worst affected countries')